In [1]:
import pandas as pd # import pandas

In [2]:
import json #import json

In [3]:
from pandas.io.json import json_normalize #import json_normalize

In [4]:
data = json.load((open('data/world_bank_projects.json')))#load the json file

In [19]:
df_mjthemes = json_normalize(data, 'mjtheme_namecode') #get a normalized dataframe of the mjtheme_namecode column

In [11]:
wb_df = pd.read_json('data/world_bank_projects.json') #turn the json file into a pandas dataframe

In [12]:
print(wb_df.columns) #examine the columns

Index(['_id', 'approvalfy', 'board_approval_month', 'boardapprovaldate',
       'borrower', 'closingdate', 'country_namecode', 'countrycode',
       'countryname', 'countryshortname', 'docty', 'envassesmentcategorycode',
       'grantamt', 'ibrdcommamt', 'id', 'idacommamt', 'impagency',
       'lendinginstr', 'lendinginstrtype', 'lendprojectcost',
       'majorsector_percent', 'mjsector_namecode', 'mjtheme',
       'mjtheme_namecode', 'mjthemecode', 'prodline', 'prodlinetext',
       'productlinetype', 'project_abstract', 'project_name', 'projectdocs',
       'projectfinancialtype', 'projectstatusdisplay', 'regionname', 'sector',
       'sector1', 'sector2', 'sector3', 'sector4', 'sector_namecode',
       'sectorcode', 'source', 'status', 'supplementprojectflg', 'theme1',
       'theme_namecode', 'themecode', 'totalamt', 'totalcommamt', 'url'],
      dtype='object')


In [15]:
wb_df_index_set = wb_df.set_index('countryname') #Turn the json file into a pandas dataframe. Set the index by country name
wb_p = wb_df_index_set['mjtheme_namecode'] #reduce the dataframe to just show the total amount for projects which is already indexed by country name
grouped1 = wb_p.groupby('countryname').count() #group the reduced dataframe by countryname and count the values of the mjtheme_namecode dolumn, which is the only column.


In [16]:
print(grouped1.head()) #Examine the results

countryname
Africa                    11
Antigua and Barbuda        1
Arab Republic of Egypt     2
Argentine Republic         2
Bosnia and Herzegovina     1
Name: mjtheme_namecode, dtype: int64


In [17]:
grouped_sorted = grouped1.sort_values(ascending=False) #sort the dataframe by descending order to get the highest values
grouped_sorted = grouped_sorted.drop('Africa') #Remove Africa from the list because it is a continent and not a country.

In [18]:
print(grouped_sorted.head(10)) #Top ten countries with the most projects. Problem 1 solved

countryname
People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Nepal                              12
People's Republic of Bangladesh    12
Kingdom of Morocco                 12
Republic of Mozambique             11
Burkina Faso                        9
Name: mjtheme_namecode, dtype: int64


In [23]:
#problem 2

In [26]:
print(df_mjthemes.head())

  code                                   name
0    8                      Human development
1   11                                       
2    1                    Economic management
3    6  Social protection and risk management
4    5                  Trade and integration


In [44]:
count_by_theme = df_mjthemes.groupby(['code', 'name']).size().reset_index(name='count').sort_values(by='count', ascending=False) #grouped the normalized dataframe by 'code' and 'name' aggregating by size, reset the index calling the aggregated column 'count'. and then sorted the values by 'count' by descending order. 

In [45]:
print(count_by_theme) #Problem 2 solved.

   code                                          name  count
5    11  Environment and natural resources management    223
3    10                             Rural development    202
19    8                             Human development    197
7     2                      Public sector governance    184
15    6         Social protection and risk management    158
11    4      Financial and private sector development    130
17    7                   Social dev/gender/inclusion    119
13    5                         Trade and integration     72
21    9                             Urban development     47
1     1                           Economic management     33
4    11                                                   27
10    4                                                   16
6     2                                                   15
2    10                                                   14
18    8                                                   13
9     3                 

In [46]:
#Problem 3

In [57]:
df_mjthemes_map = df_mjthemes.groupby('code')['name'].describe().reset_index() #get a description of the code column against the name column. reset the index

In [58]:
print(df_mjthemes_map)

   code count unique                                           top freq
0     1    38      2                           Economic management   33
1    10   216      2                             Rural development  202
2    11   250      2  Environment and natural resources management  223
3     2   199      2                      Public sector governance  184
4     3    15      2                                   Rule of law   12
5     4   146      2      Financial and private sector development  130
6     5    77      2                         Trade and integration   72
7     6   168      2         Social protection and risk management  158
8     7   130      2                   Social dev/gender/inclusion  119
9     8   210      2                             Human development  197
10    9    50      2                             Urban development   47


In [62]:
df_mjthemes_map = df_mjthemes_map[['code', 'top']].rename(columns={'top':'name'}) #rename the columns

In [63]:
print(df_mjthemes_map)

   code                                          name
0     1                           Economic management
1    10                             Rural development
2    11  Environment and natural resources management
3     2                      Public sector governance
4     3                                   Rule of law
5     4      Financial and private sector development
6     5                         Trade and integration
7     6         Social protection and risk management
8     7                   Social dev/gender/inclusion
9     8                             Human development
10    9                             Urban development


In [64]:
df_mjthemes_fill = df_mjthemes[["code"]].merge(df_mjthemes_map,
how="left", on="code") #merge the two datafarmes by the 'code' column in the left dataframe

In [68]:
print(df_mjthemes_fill.head(15)) #examine the results

   code                                          name
0     8                             Human development
1    11  Environment and natural resources management
2     1                           Economic management
3     6         Social protection and risk management
4     5                         Trade and integration
5     2                      Public sector governance
6    11  Environment and natural resources management
7     6         Social protection and risk management
8     7                   Social dev/gender/inclusion
9     7                   Social dev/gender/inclusion
10    5                         Trade and integration
11    4      Financial and private sector development
12    6         Social protection and risk management
13    6         Social protection and risk management
14    2                      Public sector governance


In [69]:
count_by_theme = df_mjthemes_fill.groupby(["code","name"]).size() \
                    .reset_index(name="count") \
                    .sort_values("count", ascending=False) #reapply the code from problem 2 now with the new filled dataframe 

In [71]:
print(count_by_theme) #Top 10 projects with the nan values correctly filled. 

   code                                          name  count
2    11  Environment and natural resources management    250
1    10                             Rural development    216
9     8                             Human development    210
3     2                      Public sector governance    199
7     6         Social protection and risk management    168
5     4      Financial and private sector development    146
8     7                   Social dev/gender/inclusion    130
6     5                         Trade and integration     77
10    9                             Urban development     50
0     1                           Economic management     38
4     3                                   Rule of law     15
